In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from src.models.model_trainer_rl_v5 import ModelTrainerRL, TradingEnvRL
from src.utils.config_loader import load_config
from src.models.backtester import PortfolioBacktester, PortfolioBacktesterRL
#from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load configuration
config = load_config("config/config.yaml")

d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\kaleido\scopes\plotly.py:32: DeprecationWarning:


Use of plotly.io.kaleido.scope.default_format is deprecated and support will be removed after September 2025.
Please use plotly.io.defaults.default_format instead.


d:\MSDS\buy-sell-hold-strategy-prediction\venv-3.11\Lib\site-packages\kaleido\scopes\plotly.py:33: DeprecationWarning:




In [2]:


# Load and prepare data
data = pd.read_csv('data/processed/CWCO_processed_2022.csv')
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

# Split into train/test
split_idx = int(len(data) * 0.7)
df_train = data.iloc[:split_idx].copy()
df_test = data.iloc[split_idx:].copy()

# Initialize trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

# Prepare environments (handles feature scaling automatically)
env_train, env_test = trainer.prepare_environment(
    df_train, 
    df_test, 
    reward_func="profit"  # or "profit", "sortino", "cvar", "max_drawdown"
)

# Train PPO only
print("Training PPO...")
result = trainer.train_ppo(env_train)
model = result["model"]
trainer.models['ppo'] = model # Register model
print("Training Complete!")

# Evaluate with multiple seeds (recommended)
# 2. Evaluate (Auto-handles normalization)
seeds = [42, 43, 44]
metrics = trainer.evaluate_over_seeds(model, env_test, seeds, algorithm="ppo")
print("Evaluation:", metrics)

print("\n--- Multi-Seed Evaluation Results ---")
print(f"Total Return: {metrics['total_return']['mean']:.2%} ± {metrics['total_return']['ci95']}")
print(f"Sharpe Ratio: {metrics['sharpe_ratio']['mean']:.3f} ± {metrics['sharpe_ratio']['ci95']}")
print(f"Sortino Ratio: {metrics['sortino_ratio']['mean']:.3f} ± {metrics['sortino_ratio']['ci95']}")
print(f"Max Drawdown: {metrics['max_drawdown']['mean']:.2%} ± {metrics['max_drawdown']['ci95']}")

# Single run evaluation (if you only want one)
metrics_single = trainer.evaluate_model(model, env_test, algorithm="PPO", seed=42)
print("\n--- Single Run Evaluation ---")
print(f"Total Return: {metrics_single['total_return']:.2%}")
print(f"Sharpe Ratio: {metrics_single['sharpe_ratio']:.3f}")

# 3. Save

trainer.save_models("models/ppo_artifacts")

# --- 4. CORRECT INFERENCE (THE FIX) ---
# Use the NEW method to get a wrapper environment that knows the training stats
env_inference = trainer.load_inference_env(df_test, algorithm="ppo")

obs = env_inference.reset()
done = False
actions = []

print("Generating actions with normalized inputs...")
while True:
    action, _ = model.predict(obs, deterministic=True)
    obs, _, dones, _ = env_inference.step(action)
    actions.append(float(action[0][0]))
    if dones[0]: break

print(f"\nGenerated {len(actions)} trading actions")

# --- CELL 5: RUN BACKTEST ---
print("Running Backtest...")

# Unpack the config dictionary using **
env_params = config['reinforcement_learning']['environment']

backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=df_test['close'], 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30) # Important for data alignment
)
comparison = backtester.compare_with_buy_and_hold_rl()
# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_37152\2309604206.py:4: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`



TypeError: src.models.model_trainer_rl_v5.TradingEnvRL() got multiple values for keyword argument 'max_steps'

In [4]:

# 1. Load Configuration
config = load_config("config/config.yaml")

# 2. Load and Prepare Data
data = pd.read_csv('data/processed/AAPL_processed_2022.csv')
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

# Split into train/test
split_idx = int(len(data) * 0.7)
df_train = data.iloc[:split_idx].copy()
df_test = data.iloc[split_idx:].copy()

# 3. Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

# Prepare environments (Raw envs, scaling handled internally by PPO)
env_train, env_test = trainer.prepare_environment(
    df_train, 
    df_test, 
    reward_func="profit" 
)

# 4. Train PPO
print("Training PPO...")
result = trainer.train_ppo(env_train)
model = result["model"]
trainer.models["ppo"] = model  # <--- CRITICAL: Register model so it gets saved later
print("Training Complete!")

# 5. Evaluate (Using the built-in method which handles normalization)
# This uses the same logic as training, so results are accurate
seeds = [42]
metrics_agg = trainer.evaluate_over_seeds(model, env_test, seeds, algorithm="PPO")

print("\n--- Multi-Seed Evaluation Results ---")
print(f"Total Return: {metrics_agg['total_return']['mean']:.2%} ± {metrics_agg['total_return']['ci95']}")
print(f"Sharpe Ratio: {metrics_agg['sharpe_ratio']['mean']:.3f} ± {metrics_agg['sharpe_ratio']['ci95']}")
print(f"Max Drawdown: {metrics_agg['max_drawdown']['mean']:.2%} ± {metrics_agg['max_drawdown']['ci95']}")

# 6. Save Artifacts
# This saves the model AND the "ppo_vecnormalize.pkl" stats file needed for inference
trainer.save_models("models/ppo_artifacts")

# ==============================================================================
# 7. INFERENCE & BACKTEST (THE FIXED PART)
# ==============================================================================

print("\n--- Generating Actions for Backtest ---")

# CRITICAL FIX: Load the "Inference Environment"
# This automatically wraps your data with the saved Mean/Std from training.
# Without this, the model sees "alien" data and panics (sells everything).
env_inference = trainer.load_inference_env(df_test, algorithm="ppo")

obs = env_inference.reset()
done = False
actions = []

# Note: Vectorized environments return done as an array [False]
while True:
    action, _ = model.predict(obs, deterministic=True)
    obs, _, dones, _ = env_inference.step(action)
    
    # Extract the single action value
    actions.append(float(action[0][0]))
    
    if dones[0]:
        break

print(f"Generated {len(actions)} trading actions")

# 8. Run Backtest
print("Running Backtest...")

env_params = config['reinforcement_learning']['environment']

# Truncate price data to match action length (VectorBT strictness)
# Sometimes VecEnv cuts the last frame, so we align lengths.
aligned_prices = df_test['close'].iloc[:len(actions)]

backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=aligned_prices, 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30)
)
comparison = backtester.compare_with_buy_and_hold_rl()

# Show Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Plot
portfolio.plot().show()

C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_37152\2669133266.py:7: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v4:Prepared training env with 648 steps (scaled=True)
INFO:src.models.model_trainer_rl_v4:Prepared testing env with 279 steps (scaled=True)


Training PPO...


INFO:src.models.model_trainer_rl_v4:Training PPO for 200000 timesteps
INFO:src.models.model_trainer_rl_v4:Evaluation - Return: -20.70%, Sharpe: -0.766, Sortino: -0.891, Max DD: -41.55%
INFO:src.models.model_trainer_rl_v4:Multi-seed evaluation (n=1): Sharpe -0.766 ± CI(-0.766, -0.766)
INFO:src.models.model_trainer_rl_v4:Saved ppo model to models/ppo_artifacts\ppo_model
INFO:src.models.model_trainer_rl_v4:Saved results summary to models/ppo_artifacts\rl_results_summary.json


Training Complete!

--- Multi-Seed Evaluation Results ---
Total Return: -20.70% ± [-0.20704288092987147, -0.20704288092987147]
Sharpe Ratio: -0.766 ± [-0.7657962747198097, -0.7657962747198097]
Max Drawdown: -41.55% ± [-0.4154851988495671, -0.4154851988495671]

--- Generating Actions for Backtest ---


AttributeError: 'ModelTrainerRL' object has no attribute 'load_inference_env'

In [2]:


# ==============================================================================
# 1. SETUP & DATA LOADING
# ==============================================================================

# Load Configuration
config = load_config("config/config.yaml")

# Load Data
data_path = 'data/processed/AAPL_processed_2022.csv'
print(f"Loading data from {data_path}...")
data = pd.read_csv(data_path)

if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

# Split into train/test
split_idx = int(len(data) * 0.7)
df_train = data.iloc[:split_idx].copy()
df_test = data.iloc[split_idx:].copy()

print(f"Train size: {len(df_train)}, Test size: {len(df_test)}")

# ==============================================================================
# 2. TRAINING
# ==============================================================================

# Initialize Trainer
trainer = ModelTrainerRL(config['reinforcement_learning'])

# Prepare environments (Raw envs, scaling handled internally by PPO)
env_train, env_test = trainer.prepare_environment(
    df_train, 
    df_test, 
    reward_func="profit" 
)

# Train PPO
print("\n--- Starting PPO Training ---")
result = trainer.train_ppo(env_train)
model = result["model"]

# CRITICAL: Register model so it gets saved later
trainer.models["ppo"] = model  
print("Training Complete!")

# ==============================================================================
# 3. EVALUATION (Internal Metrics)
# ==============================================================================

# Evaluate using multiple seeds to check stability
seeds = [42, 43, 44]
print(f"\n--- Evaluating over {len(seeds)} seeds ---")
metrics_agg = trainer.evaluate_over_seeds(model, env_test, seeds, algorithm="PPO")

print(f"Total Return: {metrics_agg['total_return']['mean']:.2%} (Range: {metrics_agg['total_return']['ci95']})")
print(f"Sharpe Ratio: {metrics_agg['sharpe_ratio']['mean']:.3f}")
print(f"Max Drawdown: {metrics_agg['max_drawdown']['mean']:.2%}")

# Save Artifacts (Model + Normalization Stats)
trainer.save_models("models/ppo_artifacts")
print("\nModels and Normalization stats saved to 'models/ppo_artifacts'")

# ==============================================================================
# 4. INFERENCE & BACKTEST (The Fix)
# ==============================================================================

print("\n--- Generating Actions for Backtest ---")

# CRITICAL FIX: Load the "Inference Environment"
# This wraps your test data with the EXACT mean/std deviation calculated during training.
env_inference = trainer.load_inference_env(df_test, algorithm="ppo")

obs = env_inference.reset()
done = False
actions = []

# Inference Loop
while True:
    # We use deterministic=True for backtesting (best reliable action)
    action, _ = model.predict(obs, deterministic=True)
    
    # Step the environment
    obs, _, dones, _ = env_inference.step(action)
    
    # Extract the single action value (PPO outputs an array)
    actions.append(float(action[0][0]))
    
    if dones[0]:
        break

print(f"Generated {len(actions)} trading actions")

# ==============================================================================
# 5. RUN PORTFOLIO BACKTEST
# ==============================================================================
print("\n--- Running VectorBT Backtest ---")

env_params = config['reinforcement_learning']['environment']

# Align prices with actions (prevent length mismatch errors)
aligned_prices = df_test['close'].iloc[:len(actions)]

backtester = PortfolioBacktesterRL(env_params) 

portfolio = backtester.run_backtest(
    price_data=aligned_prices, 
    predicted_weights=np.array(actions),
    lookback_window=env_params.get('lookback_window', 30)
)

# Benchmark Comparison
comparison = backtester.compare_with_buy_and_hold_rl()

# Show Final Metrics
metrics = backtester.get_performance_metrics()
print("\n--- Final Strategy Performance ---")
for k, v in metrics.items():
    print(f"{k}: {v:.4f}")

# Visualization
portfolio.plot().show()

Loading data from data/processed/AAPL_processed_2022.csv...
Train size: 648, Test size: 279

--- Starting PPO Training ---


C:\Users\BOOKLAPTOP\AppData\Local\Temp\ipykernel_39216\1665381518.py:14: FutureWarning:

In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`

INFO:src.models.model_trainer_rl_v5:Training PPO for 200000 timesteps
INFO:src.models.model_trainer_rl_v5:Loaded VecNormalize stats for PPO


Training Complete!

--- Evaluating over 3 seeds ---
Total Return: 0.00% (Range: [0.0, 0.0])
Sharpe Ratio: 0.000
Max Drawdown: 0.00%


INFO:src.models.model_trainer_rl_v5:Saved ppo stats to models/ppo_artifacts\ppo_vecnormalize.pkl
INFO:src.models.model_trainer_rl_v5:Loaded VecNormalize stats for ppo
INFO:BacktesterRL:Preparing Backtest. Raw Prices: 217, Predictions: 217
INFO:BacktesterRL:Running vectorbt simulation...



Models and Normalization stats saved to 'models/ppo_artifacts'

--- Generating Actions for Backtest ---
Generated 217 trading actions

--- Running VectorBT Backtest ---


INFO:BacktesterRL:Backtest successfully completed.
INFO:BacktesterRL:Strategy Return: -23.20%
INFO:BacktesterRL:Buy & Hold Return: 0.19%
INFO:BacktesterRL:Outperformance: -23.40%



--- Final Strategy Performance ---
Total Return (%): -23.2000
Annual Return (%): -35.8600
Sharpe Ratio: -1.0033
Sortino Ratio: -1.4018
Max Drawdown (%): -29.6000
Calmar Ratio: -1.2116
Win Rate (%): 44.1200
Total Trades: 68.0000
Final Value ($): 76795.2600
